In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv("../../data/processed/phemernr2_dataset_with_tvt.csv")
data.head()

,tweet_id,tweet_text,label,label2,topic,tvt,cv_fold
0,552833795142209536,The East London Mosque would like to offer its...,non-rumours,non-rumours,charliehebdo-all-rnr-threads,test,2
1,580318210609696769,BREAKING - A Germanwings Airbus A320 plane rep...,rumours,true,germanwings-crash-all-rnr-threads,training,3
2,552798891994009601,Reports that two of the dead in the #CharlieHe...,rumours,true,charliehebdo-all-rnr-threads,test,2
3,576790814942236672,After #Putin disappeared Russian TV no longer ...,non-rumours,non-rumours,putinmissing-all-rnr-threads,test,2
4,499678822598340608,Saw #Ferguson for myself. #justiceformichaelbr...,non-rumours,non-rumours,ferguson-all-rnr-threads,training,3


In [2]:
length = data.shape[0]
length

6425

In [3]:
import random

random.seed(37)

weight = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1]
tt = []
for i in range(length):
    gacha = random.sample(weight, 1)[0]
    if gacha == 0:
        tt.append("training")
    else:
        tt.append("test")

print(f"Training : {tt.count('training')} - {round(tt.count('training')/length, 2)}")
print(f"Test : {tt.count('test')} - {round(tt.count('test')/length, 2)}")

Training : 4500 - 0.7
Test : 1925 - 0.3


In [4]:
data['tt'] = pd.Series(tt)
data.head()

,tweet_id,tweet_text,label,label2,topic,tvt,cv_fold,tt
0,552833795142209536,The East London Mosque would like to offer its...,non-rumours,non-rumours,charliehebdo-all-rnr-threads,test,2,test
1,580318210609696769,BREAKING - A Germanwings Airbus A320 plane rep...,rumours,true,germanwings-crash-all-rnr-threads,training,3,training
2,552798891994009601,Reports that two of the dead in the #CharlieHe...,rumours,true,charliehebdo-all-rnr-threads,test,2,test
3,576790814942236672,After #Putin disappeared Russian TV no longer ...,non-rumours,non-rumours,putinmissing-all-rnr-threads,test,2,test
4,499678822598340608,Saw #Ferguson for myself. #justiceformichaelbr...,non-rumours,non-rumours,ferguson-all-rnr-threads,training,3,training


In [5]:
label_count = data['label'].value_counts()
label_count

non-rumours    4023
rumours        2402
Name: label, dtype: int64

In [6]:
combination = data.apply(lambda row: f"{row['label']}_{row['tt']}", axis=1).value_counts()
comparison = {}
for k, comb in combination.items():
    cv_fold = k.split("_")[1]
    label = k.split("_")[0]

    if cv_fold not in comparison:
        comparison[cv_fold] = {}
    
    comparison[cv_fold][label] = comb

labels = data['label'].unique().tolist()

def label_ratio(label_dict, labels):
    total = sum([v for k, v in label_dict.items()])
    
    report = ""
    for l in labels:
        report += f"{round(label_dict[l]/total, 2)}\t"
    
    return report[:-1]

print(f"Origin : {label_ratio(label_count, labels)}")
for cv, comp in comparison.items():
    print(f"Part-{cv} : {label_ratio(comp, labels)}")

Origin : 0.63	0.37
Part-training : 0.62	0.38
Part-test : 0.64	0.36


In [8]:
data.to_csv("../../data/processed/phemernr2_dataset_with_tvt.csv", index=False)